In [ ]:
!pip install pyspark


----
### INCOME

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("landingtoformatted").getOrCreate()

#change the distribucio o  with the accent.
landing_zone = 'landing_zone'
formatted_zone = 'formatted_zone'

datasets = [
    "landing_zone/income/2014_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2015_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2016_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2017_Distribucio_territorial_renda_familiar.csv"
]
income_df = spark.read.csv(datasets[0], header=True, inferSchema=True,nullValue ="-")

for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True, nullValue ="-")
    income_df = income_df.union(df_next)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/20 13:26:16 WARN Utils: Your hostname, MacBook-Air-de-Pol.local, resolves to a loopback address: 127.0.0.1; using 192.168.18.62 instead (on interface en0)
25/06/20 13:26:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/20 13:26:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49693)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/site-packages/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/opt/anaconda3/envs/DS_enviroment/lib/python3.12/site-packages/pyspark/accumulators.py", line 271, in poll
 

In [2]:
income_df.show(100)

+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
| Any|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Població|Índex RFD Barcelona = 100|
+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
|2014|             1|       Ciutat Vella|         1|            el Raval|   48471|                     65.9|
|2014|             1|       Ciutat Vella|         2|      el Barri Gòtic|   15911|                     98.5|
|2014|             1|       Ciutat Vella|         3|      la Barceloneta|   15181|                     84.5|
|2014|             1|       Ciutat Vella|         4|Sant Pere, Santa ...|   22674|                     92.5|
|2014|             2|           Eixample|         5|       el Fort Pienc|   31785|                    104.5|
|2014|             2|           Eixample|         6|  la Sagrada Família|   51562|                     92.4|
|2014|             

Standarize, eliminate the NULL columns

In [3]:
#we filter the data to remove rows with null values in the 'Índex RFD Barcelona = 100' column or the Names
from pyspark.sql.functions import col

filt_income = income_df.filter(
    ~(
        col('`Índex RFD Barcelona = 100`').isNull() |
        (col('Nom_districte') == 'No consta')
     )
)

----
### Incidencies

In [4]:
datasets = [
    "landing_zone/incidences/2014_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2015_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2016_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2017_Peticions_ciutadanes.csv"
]
incidences_df = spark.read.csv(datasets[0], header=True, inferSchema=True)
for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True)
    incidences_df = incidences_df.union(df_next)

In [5]:
incidences_df.show(100)

+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-------------+------------------+---------+-------------+-------------+-------+--------------------+----------+-----+-----------------+------------------+
|CODI_DISTRICTE|          DISTRICTE|ANY_DATA_TANCAMENT|              DETALL|               BARRI|FITXA_ID|    LATITUD|                AREA|COORDENADA_Y|COORDENADA_X|   LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|             ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA|ANY_DATA_ALTA|MES_DATA_ALTA| SUPORT|              CARRER|CODI_BARRI|  _id|            TIPUS|DIA_DATA_TANCAMENT|
+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+------

25/06/20 13:26:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
from pyspark.sql.functions import col, sum
incidences_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in incidences_df.columns]).show()

+--------------+---------+------------------+------+-----+--------+-------+----+------------+------------+--------+-------------+---------------+------+-------+-------------+------------------+---------+-------------+-------------+------+------+----------+---+-----+------------------+
|CODI_DISTRICTE|DISTRICTE|ANY_DATA_TANCAMENT|DETALL|BARRI|FITXA_ID|LATITUD|AREA|COORDENADA_Y|COORDENADA_X|LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA|ANY_DATA_ALTA|MES_DATA_ALTA|SUPORT|CARRER|CODI_BARRI|_id|TIPUS|DIA_DATA_TANCAMENT|
+--------------+---------+------------------+------+-----+--------+-------+----+------------+------------+--------+-------------+---------------+------+-------+-------------+------------------+---------+-------------+-------------+------+------+----------+---+-----+------------------+
|           123|      123|                 0|     0|  123|       0|    142|   0|         142|         142|     142|            0|             

In [7]:
filt_incidences = incidences_df.filter(~(col('CODI_DISTRICTE').isNull() & col('DISTRICTE').isNull()))

In [8]:
# we need to rename the columns to match the income_df
# We change the column names ANY_DATA_ALTA, to ANY so we can match later the year where the incident happened.
filt_incidences = (filt_incidences.withColumnRenamed('CODI_BARRI', 'Codi_Barri')
        .withColumnRenamed('CODI_DISTRICTE', 'Codi_Districte').withColumnRenamed('ANY_DATA_ALTA', 'Any'))

with renaming the columns we facilitate the later the unification of the tables

In [9]:
filt_incidences.show(100)

+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-------------+------------------+---------+----+-------------+--------------+--------------------+----------+---+-----------+------------------+
|Codi_Districte|          DISTRICTE|ANY_DATA_TANCAMENT|              DETALL|               BARRI|FITXA_ID|    LATITUD|                AREA|COORDENADA_Y|COORDENADA_X|   LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|             ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA| Any|MES_DATA_ALTA|        SUPORT|              CARRER|Codi_Barri|_id|      TIPUS|DIA_DATA_TANCAMENT|
+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-----

-----
### Population

In [10]:
datasets = [
    "landing_zone/population_by_geographical/2014_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2015_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2016_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2017_pad_mdb_nacionalitat-regio_sexe.json"
]
population_df = spark.read.json(datasets[0])
for dataset in datasets[1:]:
    df_next = spark.read.json(dataset)
    population_df = population_df.union(df_next)

In [11]:
population_df.show(100)

+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|Codi_Barri|Codi_Districte|Data_Referencia|NACIONALITAT_REGIO|     Nom_Barri|Nom_Districte|SEXE|Valor|
+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   1|   ..|
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   2|   ..|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   1|    6|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   2|    8|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   1|  682|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   2|  989|
|         1|             1|     2014-01-01|                 4|      el Ra

In [12]:
import pandas as pd
csv = pd.read_csv('landing_zone/pad_dimensions.csv')

In [ ]:
csv

In [13]:
population_df.columns

['Codi_Barri',
 'Codi_Districte',
 'Data_Referencia',
 'NACIONALITAT_REGIO',
 'Nom_Barri',
 'Nom_Districte',
 'SEXE',
 'Valor']

In [14]:
csv = spark.createDataFrame(csv)

In [15]:
csv

DataFrame[Codi_Dimensio: bigint, Desc_Dimensio: string, Codi_Valor: bigint, Desc_Valor_CA: string, Desc_Valor_ES: string, Desc_Valor_EN: string]

In [17]:
from pyspark.sql import functions as F

cols_to_map = ['SEXE', 'NACIONALITAT_REGIO']

for col_name in cols_to_map:
    # diccionario código → etiqueta
    mapping_dict = (csv.filter(F.col('Desc_Dimensio') == col_name)
                      .select('Codi_Valor', 'Desc_Valor_EN')
                      .where(F.col('Codi_Valor').isNotNull())
                      .rdd.map(lambda r: (int(r['Codi_Valor']), r['Desc_Valor_EN']))
                      .collectAsMap())

    expr = None
    for code, label in mapping_dict.items():
        cond = F.col(col_name) == code              # ambos bigint
        expr = F.when(cond, F.lit(label)) if expr is None else expr.when(cond, F.lit(label))

    expr = expr.otherwise(F.col(col_name).cast("string"))   # <-- rama else string

    population_df = population_df.withColumn(f"{col_name}_desc", expr)

In [18]:
from pyspark.sql import functions as F
population_df = population_df.withColumn("Any", F.substring("Data_Referencia", 1, 4).cast("int"))

In [19]:
dim_names = [row['Desc_Dimensio'] for row in csv.select('Desc_Dimensio').distinct().collect()]

In [20]:
dim_names

['LLOC_NAIX',
 'EDAT_1',
 'SEXE',
 'EDAT_Q',
 'LLOC_NAIX_CCAA',
 'NIV_EDUCA_esta',
 'NACIONALITAT_REGIO',
 'NACIONALITAT_G',
 'DECADA',
 'LLOC_NAIX_REGIO',
 'LLOC_NAIX_PAIS',
 'NACIONALITAT_PAIS',
 'LLOC_NAIX_CONTINENT',
 'N_PERSONES_AGG',
 'NACIONALITAT_CONTINENT',
 'NACIONALITAT_DOMICILI',
 'NACIONALITAT_INI',
 'DOM_65_99',
 'DOM_HOMES',
 'DOM_DONES',
 'DOM_00_18',
 'TIPUS_DOMICILI',
 'DOM_18_64',
 'CODI_BARRI_DEST',
 'CONTINENT_MOV',
 'NACIONALITAT_DEST']

In [21]:
population_df

DataFrame[Codi_Barri: bigint, Codi_Districte: bigint, Data_Referencia: string, NACIONALITAT_REGIO: bigint, Nom_Barri: string, Nom_Districte: string, SEXE: bigint, Valor: string, SEXE_desc: string, NACIONALITAT_REGIO_desc: string, Any: int]

We merged the two datasets so that, instead of having encoded values in the 'SEXE' or 'NACIONALITAT_REGIO' columns, we now have the actual values. To do this, we had to download an additional dataset that contained both the actual values and their corresponding encoding numbers.

In [22]:
population_df.printSchema()          # SEXE_desc y NACIONALITAT_REGIO_desc son string
population_df.select("SEXE", "SEXE_desc",
                     "NACIONALITAT_REGIO", "NACIONALITAT_REGIO_desc").show(10)


root
 |-- Codi_Barri: long (nullable = true)
 |-- Codi_Districte: long (nullable = true)
 |-- Data_Referencia: string (nullable = true)
 |-- NACIONALITAT_REGIO: long (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- SEXE: long (nullable = true)
 |-- Valor: string (nullable = true)
 |-- SEXE_desc: string (nullable = true)
 |-- NACIONALITAT_REGIO_desc: string (nullable = true)
 |-- Any: integer (nullable = true)

+----+---------+------------------+-----------------------+
|SEXE|SEXE_desc|NACIONALITAT_REGIO|NACIONALITAT_REGIO_desc|
+----+---------+------------------+-----------------------+
|   1|   Female|                 1|         Eastern Africa|
|   2|     Male|                 1|         Eastern Africa|
|   1|   Female|                 2|          Middle Africa|
|   2|     Male|                 2|          Middle Africa|
|   1|   Female|                 3|        Northern Africa|
|   2|     Male|                 3|        Nor

In [23]:
population_df.show(100)

+----------+--------------+---------------+------------------+--------------+-------------+----+-----+---------+-----------------------+----+
|Codi_Barri|Codi_Districte|Data_Referencia|NACIONALITAT_REGIO|     Nom_Barri|Nom_Districte|SEXE|Valor|SEXE_desc|NACIONALITAT_REGIO_desc| Any|
+----------+--------------+---------------+------------------+--------------+-------------+----+-----+---------+-----------------------+----+
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   1|   ..|   Female|         Eastern Africa|2014|
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   2|   ..|     Male|         Eastern Africa|2014|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   1|    6|   Female|          Middle Africa|2014|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   2|    8|     Male|          Middle Africa|2014|
|     

In [28]:
from pyspark.sql import functions as F

population_sel = (
    population_df
        .withColumn(
            "Valor_int",
            F.when(F.col("Valor") == "..", None)           
             .otherwise(F.col("Valor"))
             .cast("int")                                  
        )
        .select(
            "Codi_Barri",
            "Codi_Districte",
            "Nom_Barri",
            "Nom_Districte",
            "Valor_int",                                   
            "Any",
            "SEXE_desc",
            "NACIONALITAT_REGIO_desc"
        )
)


In [32]:
population_sel =population_sel.filter(F.col("Valor_int").isNotNull())

In [33]:
population_sel.show(100)

+----------+--------------+--------------------+-------------+---------+----+---------+-----------------------+
|Codi_Barri|Codi_Districte|           Nom_Barri|Nom_Districte|Valor_int| Any|SEXE_desc|NACIONALITAT_REGIO_desc|
+----------+--------------+--------------------+-------------+---------+----+---------+-----------------------+
|         1|             1|            el Raval| Ciutat Vella|        6|2014|   Female|          Middle Africa|
|         1|             1|            el Raval| Ciutat Vella|        8|2014|     Male|          Middle Africa|
|         1|             1|            el Raval| Ciutat Vella|      682|2014|   Female|        Northern Africa|
|         1|             1|            el Raval| Ciutat Vella|      989|2014|     Male|        Northern Africa|
|         1|             1|            el Raval| Ciutat Vella|       56|2014|   Female|         Western Africa|
|         1|             1|            el Raval| Ciutat Vella|      124|2014|     Male|         Western 

### Before this we have to partition the tables in the more optimal way for our analysis.

I think we should make the partition by years.

In [ ]:
filt_incidences.write.mode('overwrite').parquet("formatted_zone/incidences")
filt_income.write.mode('overwrite').parquet("formatted_zone/income")
population_sel.write.mode('overwrite').parquet("formatted_zone/population_by_geographical")


25/06/20 16:09:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 848268 ms exceeds timeout 120000 ms
25/06/20 16:09:00 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/20 16:09:02 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$